# SYDE 556/750 --- Assignment 4
**Student ID: 20823934**

*Note:* Please include your numerical student ID only, do *not* include your name.

*Note:* Refer to the [PDF](https://github.com/celiasmith/syde556-f22/raw/master/assignments/assignment_04/syde556_assignment_04.pdf) for the full instructions (including some hints), this notebook contains abbreviated instructions only. Cells you need to fill out are marked with a "writing hand" symbol. Of course, you can add new cells in between the instructions, but please leave the instructions intact to facilitate marking.

In [ ]:
# Import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Import nengo and some helper functions for Q1
import nengo
from nengo.utils.ensemble import tuning_curves
from nengo.utils.connection import eval_point_decoding

# Fix the numpy random seed for reproducible results
np.random.seed(18945)

# Some formating options
%config InlineBackend.figure_formats = ['svg']

# 1. Building an ensemble of neurons

**a) Tuning curves.** Plot the population tuning curves. Plot the representation accuracy plot ($x - \hat{x}$). Compute and report the RMSE.

In [ ]:
TAU_RC = 0.02   # 20 ms
TAU_REF = 0.002 # 2 ms
SIGMA = 20      # 0.1 * 200 Hz

In [ ]:
def q1_ensemble(radius, seed=0):
    # Generate the 1D network
    model = nengo.Network()
    with model:
        a = nengo.Ensemble(n_neurons=100,                           # 100 neurons
                           dimensions=1,                            # 1-dimensional space
                           intercepts=nengo.dists.Uniform(-1, 1),   # x-intercepts between -1 and 1
                           max_rates=nengo.dists.Uniform(100, 200), # max firing rates between 100 and 200 Hz
                           radius=radius,                           # radius
                           seed=seed)                               # random seed
        
        a_a = nengo.Connection(a, a)

    # Simulate the tuning curves
    with nengo.Simulator(model) as sim:
        tuning_x, activities = tuning_curves(a, sim)

    # Calculate the representation accuracy and sort them by eval_points
    eval_points, targets, decoded = eval_point_decoding(a_a, sim, eval_points=None)
    sorted_zipped = sorted(zip(eval_points, targets, decoded), key=lambda x: x[0])
    eval_points, targets, decoded = zip(*sorted_zipped)
    eval_points = np.array(eval_points)
    targets = np.array(targets)
    decoded = np.array(decoded)

    # Compute the RMSE
    rmse = np.sqrt(np.mean((targets - decoded)**2))

    return tuning_x, activities, targets, decoded, rmse
    
def q1a():
    tuning_x, activities, targets, decoded, rmse = q1_ensemble(radius=1, seed=0)

    # Plot the tuning curves
    plt.figure()
    plt.plot(tuning_x, activities)
    plt.title("Tuning curves for 100 neurons")
    plt.xlabel("Input x")
    plt.ylabel("Firing rate (Hz)")
    plt.show()

    # Plot the representation accuracy
    plt.figure()
    plt.plot(targets, targets - decoded)
    plt.title("Representation accuracy (x - x_hat)")
    plt.xlabel("Input x")
    plt.ylabel("x - x_hat")
    plt.show()
    
    # Report the RMSE
    print("RMSE: %0.3f" % rmse)

q1a()

**b) RMSE and radius.** Compute the RMSE for (at least) the four different radii $0.5$, $1$, $2$, and $4$. Plot your results. Make sure your neurons have the same (relative, i.e., scaled by the radius) $x$-intercepts and maximum rates across all experiments.

In [ ]:
def q1b():
    # Calulate RMSE for different radii
    radii = [2 ** power for power in range(-1, 3)]
    rmses = []
    for radius in radii:
        tuning_x, activities, targets, decoded, rmse = q1_ensemble(radius=radius, seed=0)
        rmses.append(rmse)

    # Plot RMSE vs. radius
    plt.figure()
    plt.plot(radii, rmses)
    plt.title("RMSE vs. radius")
    plt.xlabel("Radius")
    plt.ylabel("RMSE")
    plt.show()

    # Calculate y=mx+b
    m = (rmses[-1] - rmses[0]) / (radii[-1] - radii[0])
    b = rmses[0] - m * radii[0]
    print(f"RMSE = {m:.3f} * radius + {b:.3f}")
    print(f"Slope: {m:.6f}, RMSE at radius 1: {rmses[1]:.6f}")
    
q1b()

**c) Discussion.** What mathematical relationship between the radius and the RMSE do you observe (write down an equation)? Explain why this is the case.

✍ \<YOUR SOLUTION HERE\>

**d) RMSE and refractory period.** What happens to the RMSE and the tuning curves as $\tau_\mathrm{ref}$ changes between $1$ and $5\,\mathrm{ms}$? Plot the tuning curves for at least four different $\tau_\mathrm{ref}$ and produce a plot showing the RMSE over $\tau_\mathrm{ref}$. Again, make sure to use the same neuron ensemble parameters in all your trials.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**e) RMSE and membrane time constant.** What happens to the RMSE and the tuning curves as $\tau_\mathrm{RC}$ changes between $10$ and $100\,\mathrm{ms}$? Plot the tuning curves for at least four different $\tau_\mathrm{RC}$ and produce a plot showing the RMSE over $\tau_\mathrm{RC}$.  Again, make sure to use the same neuron ensemble parameters in all your trials.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**f) Discussion.** Discuss the last two results. Describe what happens to the tuning curves as $\tau_\mathrm{ref}$ and $\tau_\mathrm{RC}$ change (you do not need to come up with a mathematical relationship here). Explain why the change in tuning curve shape influences the RMSE in the way you observe.

✍ \<YOUR SOLUTION HERE\>

# 2. Connecting neurons

**a) Computing the identity function.** Show the input value and the decoded values from the two  ensembles in three separate plots. Run the simulation for $0.5\,\mathrm{s}$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**b) Computing an affine transformation.** Make a new version of the model where instead of computing the identity function, it computes $y(t) = 1 - 2x(t)$. Show the same graphs as in part (a).

In [ ]:
# ✍ <YOUR SOLUTION HERE>

# 3. Dynamics

**a) Transforming the dynamical system.** Rewrite the linear dynamical system describing the integrator in terms of $\frac{\mathrm{d}\vec x(t)}{\mathrm{d}t} = \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{u}$, i.e., write down the matrices $\mathbf{A}$ and $\mathbf{B}$ (you can just use the equations from class, you do not have to re-derive the equations) What are the matrices $\mathbf{A}'$ and $\mathbf{B}'$ we have to use when implementing this system using the recurrent connection post-synaptic filter?

✍ \<YOUR SOLUTION HERE\>

**b) Integrator using spiking neurons.**  Show the input, the ideal integral, and the value represented by the ensemble when the input is a value of $0.9$ from $t=0.04$ to $t=1.0$ (and $0$ for other times). Run the simulation for $1.5\,\mathrm{s}$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**c) Discussion.** What is the expected ideal result, i.e., if we just mathematically computed the integral of the input, what is the equation describing the integral? How does the simulated output compare to that ideal?

✍ \<YOUR SOLUTION HERE\>

**d) Simulation using rate neurons.** Change the neural simulation to rate mode. Re-run the simulation in rate mode. Show the resulting plots.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**e) Discussion.** How does this compare to the result in part (b)? What deviations from the ideal do you still observe? Where do those deviations come from?

✍ \<YOUR SOLUTION HERE\>

**f) Integration of a shorter input pulse.** Returning to spiking mode, change the input to be a value of $0.9$ from $t=0.04$ to $0.16$. Show the same plots as before (the input, the ideal, and the value represented by the ensemble over $1.5\,\mathrm{s}$).

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**g) Discussion.** How does this compare to (b)? What is the ideal equation? Does it work as intended? If not, why is it better or worse?

✍ \<YOUR SOLUTION HERE\>

**h) Input ramp.** Change the input to a ramp input from $0$ to $0.9$ from $t=0$ to $t=0.45$ (and $0$ for $t>0.45$). Show the same plots as in the previous parts of this question.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**i) Discussion.** What does the ensemble end up representing, and why? What is the (ideal) equation for the curve traced out by the ensemble?

✍ \<YOUR SOLUTION HERE\>

**j) Sinusoidal input.** Change the input to $5\sin(5t)$. Show the same plots as before.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**k) Discussion.** What should the value represented by the ensemble be? Write the equation. How well does it do? What are the differences between the model's behaviour and the expected ideal behaviour and why do these differences occur?

✍ \<YOUR SOLUTION HERE\>

**l) 🌟 Bonus question.** Implement a nonlinear dynamical system we have not seen in class (and that is not in the book). Demonstrate that it's working as expected

✍ \<YOUR SOLUTION HERE\>

In [ ]:
# ✍ <YOUR SOLUTION HERE>